## Tests & examples with pyvkfft

In [ ]:
# Use the following to install pyvkfft on google colab
if False:
    # Install pyvkfft & dependencies only once using pip
    import os
    if not os.path.exists('dev/pyvkfft'):
      !mkdir dev
      !cd dev && git clone https://github.com/DTolm/VkFFT.git
      !cd dev && git clone https://github.com/vincefn/pyvkfft.git
      !cp dev/VkFFT/vkFFT/vkFFT.h /usr/local/include/
      !pip install pycuda
      # !pip install cupy
      !cd dev/pyvkfft && pip install ./
      # scipy, matplotlib not required for pyvkfft, but for tests
      !pip install scipy matplotlib

## Select PyCUDA or CuPy

In [ ]:
try:
    # raise ImportError()  # Uncomment to force using cupy if you have both
    import pycuda.autoinit
    import pycuda.driver as cu_drv
    import pycuda.gpuarray as cua
    synchronize = cu_drv.Context.synchronize
    gpu_empty = cua.empty
    gpu_empty_like = cua.empty_like
    to_gpu = cua.to_gpu
    stream = cu_drv.Stream()
    print("Using PyCUDA")
    
except ImportError:
    try:
        import cupy
        synchronize = cupy.cuda.stream.get_current_stream().synchronize
        gpu_empty = cupy.empty
        gpu_empty_like = cupy.empty_like
        to_gpu = cupy.asarray
        stream = cupy.cuda.Stream()
        print("Using CuPy")
    except ImportError:
        raise ImportError("You need either PyCUDA or Cupy")

import pyvkfft.cuda
from pyvkfft.cuda import VkFFTApp
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.misc import ascent
import numpy as np
from scipy.fft import fftn, ifftn, fftshift, rfftn, irfftn, dctn, idctn
import timeit


In [ ]:


def speed(shape, ndim, nb=10, stream=None, inplace=True, norm=0):
    """
    Perform a speed test using VkFFT (
    :param shape: array shape to use
    :param ndim: number of dimensions for the FFT (e.g. can be 1, 2 or 3 for a 3D array, etc..)
    :param nb: number of repeats for timing
    :param stream: the pycuda.driver.Stream to be sued for calculations. If None,
        the default stream for the active context will be used.
    :param inplace: if True (default), do an in-place FFT
    :param norm: norm=0 will have the L2 norm multiplied by the FT size for each transform,
        whereas norm=1 will divide the L2 norm by the FT size for a backwards transform,
        similarly to numpy's fft norm='backwards'.
    :return: a tuple with the time per couple of FFT and iFFT, and the idealised memory throughput
        assuming one read and one write of the array per transform axis, in Gbytes/s.
    """
    d = to_gpu(np.random.uniform(0, 1, shape).astype(np.complex64))
    if not inplace:
        d1 = gpu_empty_like(d)
    # print(d.shape)
    app = VkFFTApp(d.shape, d.dtype, ndim=ndim, stream=stream, inplace=inplace, norm=norm)
    synchronize()
    t0 = timeit.default_timer()
    for i in range(nb):
        if inplace:
            d = app.ifft(d)
            d = app.fft(d)
        else:
            d1 = app.ifft(d, d1)
            d = app.fft(d1, d)
    synchronize()
    dt = timeit.default_timer() - t0
    shape = list(shape)
    if len(shape) < 3:
        shape += [1] * (3 - len(shape))
    gbps = d.nbytes * nb * ndim * 2 * 2 / dt / 1024 ** 3
    s = ""
    if not inplace:
        s= "[out-of-place]"
    print("(%4d %4d %4d)[%dD] dt=%6.2f ms %7.2f Gbytes/s %s [norm=%d]" %
          (shape[2], shape[1], shape[0], ndim, dt / nb * 1000, gbps, s, norm))
    return dt, gbps


## Speed tests

In [ ]:
if False:
    for inplace in [True, False]:
        speed((256, 256, 256), 3, inplace=inplace)
        speed((400, 400, 400), 3, inplace=inplace)
        speed((512, 512, 512), 3, inplace=inplace)
        speed((16, 504, 504), 2, inplace=inplace)
        speed((16, 512, 512), 2, inplace=inplace)
        speed((16, 1024, 1024), 2, inplace=inplace)
        speed((16, 2048, 2048), 2, inplace=inplace)
        speed((1, 512, 512), 2, inplace=inplace)
        speed((1, 1024, 1024), 2, inplace=inplace)
        speed((1, 2600, 2048), 2, inplace=inplace)
        speed((1, 2048, 2600), 2, inplace=inplace)
        speed((8, 2600, 2048), 2, inplace=inplace)
        speed((8, 2048, 2600), 2, inplace=inplace)
        speed((16, 2200, 2400), 2, inplace=inplace)
        speed((16, 2310, 2730), 2, inplace=inplace)
        speed((16, 2730, 2310), 2, inplace=inplace)  # 2310=2*3*5*7*11, 2730=2*3*5*7*13
        speed((16, 2400, 2200), 2, inplace=inplace)
        speed((1, 3080, 3080), 2, inplace=inplace)
        speed((8, 3072, 3072), 2, inplace=inplace)

        # Also test with a supplied stream
        speed((16, 1000, 1000), 2, stream=stream, inplace=inplace)


## Complex<->Complex, inplace

In [ ]:
# single precision
d = to_gpu(ascent().astype(np.complex64))
app = VkFFTApp(d.shape, d.dtype, ndim=2)

print((abs(d.get()) ** 2).sum())
d = app.fft(d)
print((abs(d.get()) ** 2).sum())

print()

d = app.ifft(d)
print((abs(d.get()) ** 2).sum())

# Double precision
d = to_gpu(ascent().astype(np.complex128))
app = VkFFTApp(d.shape, d.dtype, ndim=2)

plt.figure(figsize=(15, 5))
plt.subplot(121)
plt.imshow(np.fft.fftshift(abs(np.fft.fftn(ascent().astype(np.complex128)))), norm=LogNorm())
plt.colorbar()

print((abs(d.get()) ** 2).sum())
d = app.fft(d)

plt.subplot(122)
plt.imshow(np.fft.fftshift(abs(d.get())), norm=LogNorm())
plt.colorbar()

print((abs(d.get()) ** 2).sum())
d = app.ifft(d)
print((abs(d.get()) ** 2).sum())


## Complex<->Complex, inplace, y-axis transform
Use `axes=(-2,)` to perform a transform along specific axes, skipping the first dimension

In [ ]:
# single precision
d = to_gpu(ascent().astype(np.complex64))
axes = (-2,)
app = VkFFTApp(d.shape, d.dtype, axes=axes)

print((abs(d.get()) ** 2).sum())
d = app.fft(d)
print((abs(d.get()) ** 2).sum())

print()

d = app.ifft(d)
print((abs(d.get()) ** 2).sum())

# Double precision
d = to_gpu(ascent().astype(np.complex128))
app = VkFFTApp(d.shape, d.dtype, axes=axes)

plt.figure(figsize=(15, 5))
plt.subplot(121)
plt.imshow(np.fft.fftshift(abs(np.fft.fftn(ascent().astype(np.complex128), axes=axes))), norm=LogNorm())
plt.colorbar()

print((abs(d.get()) ** 2).sum())
d = app.fft(d)

plt.subplot(122)
plt.imshow(np.fft.fftshift(abs(d.get())), norm=LogNorm())
plt.colorbar()

print((abs(d.get()) ** 2).sum())
d = app.ifft(d)
print((abs(d.get()) ** 2).sum())


## complex<->complex, out-of-place

In [ ]:
# single precision, out-of-place
d = to_gpu(ascent().astype(np.complex64))
d1 = gpu_empty_like(d)
app = VkFFTApp(d.shape, d.dtype, inplace=False, ndim=2)

print((abs(d.get()) ** 2).sum())
plt.figure(figsize=(15, 4), dpi=100)
plt.subplot(141)
plt.imshow(ascent())
plt.title('Original array')

d1 = app.fft(d, d1)
print((abs(d1.get()) ** 2).sum())
dn = fftn(ascent().astype(np.complex64))
print(np.allclose(dn, d1.get(), rtol=1e-6,atol=abs(dn).max()*1e-6))

plt.subplot(142)
plt.imshow(abs(d.get()))
plt.title('Source array after FFT')

plt.subplot(143)
plt.imshow(fftshift(abs(d1.get())), norm=LogNorm(vmin=100))
plt.colorbar()
plt.title('FFT')

plt.subplot(144)
plt.imshow(fftshift(abs(fftn(ascent().astype(np.complex64)))), norm=LogNorm(vmin=100))
plt.colorbar()
plt.title('FFT (numpy)')


d = app.ifft(d1, d)
print((abs(d.get()) ** 2).sum())
print(np.allclose(ascent(), d.get(), rtol=1e-6,atol=ascent().max()*1e-6))

# double precision, out-of-place
d = to_gpu(ascent().astype(np.complex128))
d1 = gpu_empty_like(d)
app = VkFFTApp(d.shape, d.dtype, inplace=False, ndim=2)

print((abs(d.get()) ** 2).sum())

d1 = app.fft(d, d1)
print((abs(d1.get()) ** 2).sum())
dn = fftn(ascent().astype(np.complex128))
print(np.allclose(dn, d1.get(), rtol=1e-12,atol=abs(dn).max()*1e-12))

d = app.ifft(d1, d)
print((abs(d.get()) ** 2).sum())


## Real<->Complex (half-Hermitian) inplace transform

In [ ]:
# single precision, R2C inplace
d0 = np.zeros((512, 514), dtype=np.float32)
d0[:, :512] = ascent()

d = to_gpu(d0)
app = VkFFTApp(d.shape, d.dtype, ndim=2, r2c=True)

print((abs(d.get()) ** 2).sum())
print(d.shape, d.dtype)
d = app.fft(d)
print(d.shape, d.dtype)
print((abs(d.get()) ** 2).sum())

dn = np.fft.rfftn(d0[:, :-2].astype(np.float32))
print(dn.shape)
print(np.allclose(d.get(), dn, rtol=1e-6, atol=dn.max() * 1e-6))

plt.figure(figsize=(15,5), dpi=100)
plt.subplot(121)
plt.imshow(fftshift(abs(d.get()), axes=0), norm=LogNorm(vmin=100))
plt.colorbar()
plt.title("R->C")
plt.subplot(122)
plt.imshow(fftshift(abs(dn), axes=0), norm=LogNorm(vmin=100))
plt.colorbar()
plt.title("R->C (numpy)")

d = app.ifft(d)
print(d.shape, d.dtype)
print((abs(d.get()) ** 2).sum())

print(np.allclose(d.get()[:, :-2], d0[:, :-2], rtol=1e-6, atol = 255e-6))


## Real<->Complex (half-Hermitian) out-of-place transform

In [ ]:
# single precision, R2C inplace
d0 = to_gpu(ascent().astype(np.float32))
d1 = gpu_empty((512, 257), dtype=np.complex64)

app = VkFFTApp(d0.shape, d0.dtype, ndim=2, r2c=True, inplace=False)

print((abs(d0.get()) ** 2).sum())
print(d0.shape, d0.dtype)
d1 = app.fft(d0, d1)
print(d1.shape, d1.dtype)
print((abs(d1.get()) ** 2).sum())

dn = np.fft.rfftn(ascent().astype(np.float32))
print(dn.shape)
print(np.allclose(d1.get(), dn, rtol=1e-6, atol=dn.max() * 1e-6))

plt.figure(figsize=(15,5), dpi=100)
plt.subplot(121)
plt.imshow(fftshift(abs(d1.get()), axes=0), norm=LogNorm(vmin=100))
plt.colorbar()
plt.title("R->C out-of-place")
plt.subplot(122)
plt.imshow(fftshift(abs(dn), axes=0), norm=LogNorm(vmin=100))
plt.colorbar()
plt.title("R->C (numpy)")

d0 = app.ifft(d1, d0)
print(d0.shape, d0.dtype)
print((abs(d0.get()) ** 2).sum())

print(np.allclose(d0.get(), ascent(), rtol=1e-6, atol = 255e-6))

plt.figure(figsize=(15,5), dpi=100)
plt.subplot(121)
plt.imshow(d0.get())
plt.colorbar()
plt.title("R->C & C->R out-of-place")
plt.subplot(122)
plt.imshow(ascent())
plt.colorbar()
plt.title("Original array")


## Real<->Real transform = Direct Cosine Transform (DCT)
All forward DCT give the same result as scipy.
Only type 2 and 3 inverse transforms give the same result.

In [ ]:
# single precision, DCT inplace
for dct_type in (1,2,3,4):
    print("Testing DCT type %d" % dct_type)
    d0 = to_gpu(ascent().astype(np.float32))

    # dct type  (default if dct=True)
    app = VkFFTApp(d0.shape, d0.dtype, ndim=2, dct=dct_type, inplace=True)

    print((abs(d0.get()) ** 2).sum())
    d0 = app.fft(d0, d0)
    print((abs(d0.get()) ** 2).sum())

    dn = dctn(ascent().astype(np.float32), type=dct_type)
    print(np.allclose(d0.get(), dn, rtol=1e-6, atol=dn.max() * 1e-6))

    fig = plt.figure(figsize=(15,5), dpi=100)
    plt.subplot(121)
    plt.imshow(abs(d0.get()), norm=LogNorm(vmin=100))
    plt.colorbar()
    plt.title("DCT-%d in-place" % dct_type)
    plt.subplot(122)
    plt.imshow(abs(dn), norm=LogNorm(vmin=100))
    plt.colorbar()
    plt.title("DCT-%d (scipy)" % dct_type)
    
    # inverse transform:
    d0 = app.ifft(d0, d0)
    idn = idctn(dn, type=dct_type)
    print((abs(d0.get()) ** 2).sum(), (abs(idn) ** 2).sum())

    print(np.allclose(d0.get(), idn, rtol=1e-4, atol = 1e-3),
          np.allclose(d0.get(), ascent(), rtol=1e-4, atol = 1e-3))

    fig = plt.figure(figsize=(15,5), dpi=100)
    plt.subplot(121)
    plt.imshow(d0.get())
    plt.colorbar()
    plt.title("DCT + iDCT type %d in-place" % dct_type)
    plt.subplot(122)
    plt.imshow(ascent())
    plt.colorbar()
    plt.title("Original array")
    print()

Further normalisation tests for DCT type 1 and 4 (correction factor with vkfft 1.2.12)

In [ ]:
from scipy.misc import face
a0 = ascent().astype(np.float32)[:400,:400]
#a0 = face().astype(np.float32)[:400,:400, 0]

dct_type = 4
print("Testing DCT type %d" % dct_type)
d0 = to_gpu(a0)

# dct type  (default if dct=True)
app = VkFFTApp(d0.shape, d0.dtype, ndim=2, dct=dct_type, inplace=True)

print((abs(d0.get()) ** 2).sum())
d0 = app.fft(d0, d0)
print((abs(d0.get()) ** 2).sum())

dn = dctn(a0, type=dct_type)
print(np.allclose(d0.get(), dn, rtol=1e-6, atol=dn.max() * 1e-6))

fig = plt.figure(figsize=(15,5), dpi=100)
plt.subplot(121)
plt.imshow(abs(d0.get()), norm=LogNorm(vmin=100))
plt.colorbar()
plt.title("DCT-%d in-place" % dct_type)
plt.subplot(122)
plt.imshow(abs(dn), norm=LogNorm(vmin=100))
plt.colorbar()
plt.title("DCT-%d (scipy)" % dct_type)

# inverse transform:
d0 = app.ifft(d0, d0)
idn = idctn(dn, type=dct_type)
n = len(d0)
d2 = d0.get()
if dct_type == 1:
    d2 *= ((n)/(n-1))**2
if dct_type == 4:
    d2 *= 4

print((abs(d2) ** 2).sum(), (abs(idn) ** 2).sum())

print(np.allclose(d2, idn, rtol=1e-4, atol = 1e-3),
      np.allclose(d2, a0, rtol=1e-4, atol = 1e-3))

fig = plt.figure(figsize=(15,5), dpi=100)
plt.subplot(121)
plt.imshow(d2-a0)
plt.colorbar()
plt.title("DCT + iDCT type %d in-place" % dct_type)
plt.subplot(122)
plt.imshow(a0)
plt.colorbar()
plt.title("Original array")
print()